# 利用SqueezeNet结构训练cumt数据
## 2017年11月8日

In [ ]:
# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload
import pickle
import tensorflow as tf
import numpy as np
from cumt_eye import Cumt_itraker
import matplotlib.pyplot as plt
import cv2
from Cumt_SqueezeNet import squeezenet
init=tf.global_variables_initializer()

sess=tf.InteractiveSession()
cumt_data=np.load('cumt_data.pkl')
index=np.arange(cumt_data['data'].shape[0])
np.random.shuffle(index)
data=cumt_data['data'][index]
label=cumt_data['labels'][index]
del cumt_data

#cumt rgb 相片均值
cumt_picmean=[103.939, 116.779, 123.68]

#### 将图片从 （32,128,3）转换成（128,128,3），有可能会导致精度下降，非必要


In [ ]:
resize_img=[]
for i in range(data.shape[0]):
    img=data[i]+cumt_picmean
    img=cv2.resize(img.astype('uint8'),(128,128))
    resize_img.append(img.astype('uint8'))
    if i%10000==0:
        print (i)
data=None
resize_img=np.array(resize_img).astype('uint8')

#### 常规设置占位符，训练节点等

In [ ]:
with tf.name_scope('PlaceHolder'):
    X=tf.placeholder(shape=[None,128,128,3],dtype=tf.float32)
    Y=tf.placeholder(shape=[None,10],dtype=tf.float32)
    LR=tf.placeholder(dtype=tf.float32)
with tf.name_scope('model'):
    model=squeezenet(X,sess)
    y_score=model.score
with tf.name_scope('LOSS'):
    LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_score,labels=Y))
    tf.summary.scalar('loss',LOSS)
with tf.name_scope('TRAIN'):
    TRAIN=tf.train.AdamOptimizer(LR).minimize(LOSS)
with tf.name_scope('ACCURACY'):
    acc_count=tf.equal(tf.arg_max(y_score,1),tf.arg_max(Y,1))
    ACCURACY=tf.reduce_mean(tf.cast(acc_count,tf.float32))
    tf.summary.scalar('acc',ACCURACY)
writer_tr=tf.summary.FileWriter('./mylog/squeezenet/train')
writer_te=tf.summary.FileWriter('./mylog/squeezenet/test',sess.graph)
writer_val=tf.summary.FileWriter('./mylog/squeezenet/val')
merge=tf.summary.merge_all()

#### train,test,val数据生成

In [ ]:
data=resize_img
index=np.arange(data.shape[0])
np.random.shuffle(index)
# tr_index=index[:100]
# te_index=index[100:200]
tr_index=index[:int(data.shape[0]*0.9)]
te_index=index[int(data.shape[0]*0.9):]
tr_data,tr_label=data[tr_index],label[tr_index]
te_data,te_label=data[te_index],label[te_index]
del data
del label
del resize_img
print(tr_data.shape,te_data.shape)


val_d=np.load('valimg1k_1109.pkl')
val_data=[]
for v in range(val_d['data'].shape[0]):
    img_=cv2.resize(val_d['data'][v],(128,128))
    val_data.append(img_)
val_data=np.array(val_data)
val_label=val_d['label']
del val_d
print(val_data.shape)

#### 开始训练

In [ ]:
init=tf.global_variables_initializer()
sess.run(init)
saver=tf.train.Saver()

In [ ]:
model.extract_network_weight('squeezenet_669.pkl')

In [ ]:
import time

In [ ]:
#5k each hour
batchsize=83
#warm up,先过拟合一个小数据集，使得网络有一个比较好的初始权值(解决网络难收敛问题)
s_time=time.time()
for j in range(5000):
    mask=np.random.choice(100,batchsize,replace=False)
    x_,y_=tr_data[mask]-cumt_picmean,tr_label[mask]
    feed_dict={X:x_,Y:y_,LR:1e-3}
    sess.run(TRAIN,feed_dict=feed_dict)
    if j%10==0:
        feed_dict={X:x_,Y:y_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        #writer_tr.add_summary(m_,i)
        print('warm up,epoch:{},train loss:{},train accuracy:{}'.format(j,loss_,acc_))
        if acc_>0.9:
            print('warm up ready!time eslape:{}'.format(time.time()-s_time))
            break

#开始正式训练
lr_=1e-3
bestval_acc=0.7
for i in range(1,1000*6*12):
    mask=np.random.choice(tr_data.shape[0],batchsize,replace=False)
    x_,y_=tr_data[mask]-cumt_picmean,tr_label[mask]
    feed_dict={X:x_,Y:y_,LR:lr_}
    sess.run(TRAIN,feed_dict=feed_dict)
    if i%10==0:
        feed_dict={X:x_,Y:y_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i)
        print('epoch:{},train loss:{},train accuracy:{}'.format(i,loss_,acc_))
    if i%20==0:
        mask=np.random.choice(te_data.shape[0],batchsize,replace=False)
        x_,y_=te_data[mask]-cumt_picmean,te_label[mask]
        feed_dict={X:x_,Y:y_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_te.add_summary(m_,i)
        print('--epoch:{},test loss:{},test accuracy:{}'.format(i,loss_,acc_))
    if i%30==0:
        mask=np.random.choice(val_data.shape[0],128,replace=False)
        x_,y_=val_data[mask]-cumt_picmean,val_label[mask]
        feed_dict={X:x_,Y:y_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_val.add_summary(m_,i)
        print('@@epoch:{},val loss:{},val accuracy:{}'.format(i,loss_,acc_))
        if acc_>bestval_acc+0.1:
            bestval_acc=acc_
            model_name=r'./model_save/SqueezeNet/model_'+str(int(acc_*10000))+'.ckpt'
            saver.save(sess,model_name,global_step=i)
            print('!!!!!model save!!!!!')
    if i%1000==0:
        lr_*=0.9
        lr_=max(lr_,1e-6)
        print('epoch:{},learning rate change:{}'.format(i,lr_))

In [ ]:
#保存模型
saver=tf.train.Saver()
saver.save(sess,'./model_save/SqueezeNet/model_984.ckpt',global_step=i)

In [ ]:
saver=tf.train.Saver()
#model_984.ckpt-34925
#model_8437.ckpt-1020
saver.restore(sess,r'D:\Proj_DL\Code\Proj_EyeTraker\Proj_iTraker\CUMT_iTraker\model_save\SqueezeNet\model_984.ckpt-34925')

#### 测试在验证集上的平均正确率，并提取权值保存

In [ ]:

acc_lis=[]
for k in range(100):
    mask=np.random.choice(val_data.shape[0],128,replace=False)
    x_,y_=val_data[mask]-cumt_picmean,val_label[mask]
    feed_dict={X:x_,Y:y_}
    acc_=sess.run(ACCURACY,feed_dict=feed_dict)
    acc_lis.append(acc_)
    #print(acc_)
print('acc mean:')
np.mean(acc_lis)

In [ ]:
n=list(tf.trainable_variables())
model_d={}
for k in n:
    #变量名称
    a=str(k).split("'")[1][:-2]
    print(a)
    with tf.variable_scope('',reuse=True):
        var=tf.get_variable(a)
        model_d[a]=var.eval()
        print(tf.shape(var).eval())



In [ ]:
list(model_d.keys())

In [ ]:
fp=open('squeezenet_795.pkl','wb')
pickle.dump(obj=model_d,file=fp)
fp.close()

In [ ]:
map_lab={ 
        '7':4,'8':3,'9':2,
        '4':7,'5':6,'6':5,
        '1':10,'2':9,'3':8,
'0':1}
         

In [ ]:
prob=tf.nn.softmax(y_score)

In [ ]:
val_data=[]
for v in range(val_d['images'].shape[0]):
    img_=cv2.resize(val_d['images'][v],(128,128))
    val_data.append(img_.)

In [ ]:
x_=val_data-cumt_picmean
mean_sum=0
for k in range(50):
    mask=np.random.choice(range(val_data.shape[0]),128,replace=False)
    acc_=sess.run(ACCURACY,feed_dict={X:x_[mask],Y:val_d['labels'][mask]})
    #print(acc_)
    mean_sum+=acc_
print('mean accuracy:')
print(mean_sum/50)

In [ ]:
import os
from scipy.misc import imread
from scipy.misc import imresize

In [ ]:
def getimg(addr):
    VGG_MEAN = [103.939, 116.779, 123.68]
    a=imread(addr)
    plt.imshow(a)
    plt.show()
    b=imresize(a-VGG_MEAN,(32,128,3)).reshape((1,32,128,3))
    return b

In [ ]:
img_dir=r'D:\Proj_DL\Code\Proj_EyeTraker\eye_val\441_5.jpg'
b=getimg(img_dir)

In [ ]:
p_=sess.run(prob,feed_dict={X:b,DROPOUT:1.,BN_FLAG:False,LR:1e-3})

In [ ]:
list(p_)

In [ ]:
np.argmax(p_)+1

In [ ]:
#计算所有可训练参数的数量
sum_=0
for s in list(tf.trainable_variables()):
    s=str(s)
    shape_=s[s.index('(')+1:s.index(')')].split(',')
    s_=1
    #print(shape_)
    for i in shape_:
        if i =='':continue
        #print(i)
        s_*=int(i)
    sum_+=s_

In [ ]:
sum_